# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

#!pip install azureml-sdk==1.19.0

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
I will be using a diabetes dataset from datahub.io dataset on gestational diabetes. https://datahub.io/machine-learning/diabetes/r/diabetes.csv
This dataset will aid to forecast gestational diabetes in women.


In [3]:
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "DSBankCluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V12',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)
# For a more detailed view of current AmlCompute status, use get_status().

###################
# Define Dataset
###################
found = False
key = "prenataldiabetes"
description_text = "Capstone Project - Forecast Diabetes"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://www.kaggle.com/uciml/pima-indians-diabetes-database?select=diabetes.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

trn-aue-aa-ml
trn-aue-advanced-analytics-ml
australiaeast
8a086c9c-5530-4791-89af-62c8cdfda3fd
Found existing cluster, use it.
Succeeded....................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


,preg,plas,pres,skin,insu,mass,pedi,age
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000


In [4]:
# choose a name for experiment
experiment_name = 'automlprenataldiabetes'
project_folder = './Augusto_Avila/capstone'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

Choosing a classification model to predict the likelihood of gestational diabetes, the variable class will hold the forecasted value and will be using the 
AUC_weighed metric as the result of initial automl experiment execution as the primary metric

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="class",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [8]:
# TODO: Submit your experiment

from azureml.widgets import RunDetails
from azureml.core.run import Run

remote_run = experiment.submit(automl_config,show_output=True)


Running on remote.
No run_configuration provided, running on DSBankCluster with default configuration
Running on remote compute: DSBankCluster
Parent Run ID: AutoML_14aceda5-4c46-4098-80d8-d344fee3e887

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************



## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
print(remote_run.get_portal_url())
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

https://ml.azure.com/experiments/automlprenataldiabetes/runs/AutoML_14aceda5-4c46-4098-80d8-d344fee3e887?wsid=/subscriptions/8a086c9c-5530-4791-89af-62c8cdfda3fd/resourcegroups/trn-aue-advanced-analytics-ml/workspaces/trn-aue-aa-ml


****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanc

{'runId': 'AutoML_14aceda5-4c46-4098-80d8-d344fee3e887',
 'target': 'DSBankCluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-20T20:51:18.82635Z',
 'endTimeUtc': '2021-01-20T21:20:28.238825Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'DSBankCluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automlprenataldiabetes","subscription_id":"8a086c9c-5530-4791-89af-62c8cdfda3fd","resource_group":"trn-aue-advanced-analytics-ml","workspace_name":"trn-aue-aa-ml","region":"australiaeast","compute_target":"DSBankCluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":nul

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
import joblib
import azureml.train.automl

# Get your best run and save the model from that run.

#!pip install azureml-train-automl-runtime==1.20.0

if "output" not in os.listdir():
   os.mkdir("./output")   

remote_run.get_metrics('AUC_weighted')

best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

#best_run = remote_run.get_best_child()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print ('Best run id:',best_run.id)
print ('################################')
print ('\n Accuracy:',best_run_metrics['AUC_weighted'])
print ('################################')
print ('\n Learning rate:', parameter_values)
print ('################################')


best_run.get_details()
best_run.get_tags()


Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


Run(Experiment: automlprenataldiabetes,
Id: AutoML_14aceda5-4c46-4098-80d8-d344fee3e887_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_split=0.15052631578947367,
                                                                                                  min_weight_fraction_l

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":3,"CurrentNodeCount":5}',
 'ensembled_iterations': '[1, 29, 21, 26, 31, 2, 6, 16]',
 'ensembled_algorithms': "['XGBoostClassifier', 'LightGBM', 'XGBoostClassifier', 'LightGBM', 'LightGBM', 'RandomForest', 'ExtremeRandomTrees', 'ExtremeRandomTrees']",
 'ensemble_weights': '[0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.2222222222222222]',
 'best_individual_pipeline_score': '0.8366210317228242',
 'best_individual_iteration': '1',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_14aceda5-4c46-4098-80d8-d344fee3e887_ModelExplain',
 'model_explanation': 'True'}

In [17]:

best_run.get_file_names()
joblib.dump(parameter_values,filename='./output/Capstone_automl_best.joblib')

['./output/Capstone_automl_best.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service